In [24]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# Muat model deteksi ekspresi wajah
model = load_model('emotion_model.h5')  # Pastikan Anda memiliki model deteksi ekspresi wajah

# Muat dataset movies.csv untuk rekomendasi
movies_df = pd.read_csv('emotion_service/dataset/processed_movies.csv')

# Daftar ekspresi yang dapat dikenali
emotion_labels = ['Angry', 'Disguist', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Muat model deteksi wajah
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Fungsi untuk merekomendasikan film berdasarkan ekspresi
def recommend_movies(emotion):
    recommendations = {
        'Happy': ['Comedy', 'Romantic', 'Action'],
        'Sad': ['Comedy','Drama', 'Romantic'],
        'Angry': ['Action', 'Thriller','Comedy'],
        'Fear': ['Horror', 'Mystery'],
        'Surprise': ['Fantasy', 'Action','Thriller'],
        'Neutral': ['Adventure', 'Animation'],
        'Disguist': ['Horror', 'Thriller']
    }

    genres = recommendations.get(emotion, ['General'])
    recommended_movies = movies_df[movies_df['genres'].isin(genres)]
    return recommended_movies[['title', 'genres']]

# Fungsi untuk mendeteksi wajah dan ekspresi
def detect_emotion_from_face():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            face = frame[y:y + h, x:x + w]
            # Mengubah wajah menjadi RGB dengan tiga saluran warna
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            
            # Pastikan dimensi gambar sesuai dengan yang diinginkan model (misalnya 48x48x3)
            face = cv2.resize(face, (48, 48))
            
            # Normalisasi gambar ke rentang [0, 1]
            face = face / 255.0
            
            # Menambahkan dimensi untuk batch dan channel terakhir (48, 48, 3)
            face = np.expand_dims(face, axis=0)
            
            # Prediksi ekspresi wajah
            emotion_prob = model.predict(face)
            max_index = np.argmax(emotion_prob[0])
            emotion = emotion_labels[max_index]


            # Menampilkan ekspresi wajah dan rekomendasi film
            recommended_movies = recommend_movies(emotion)
            print(f"Detected Emotion: {emotion}")
            print("Recommended Movies:")
            print(recommended_movies)
    
        cap.release()

# Menjalankan fungsi deteksi ekspresi wajah
if __name__ == "__main__":
    detect_emotion_from_face()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Detected Emotion: Happy
Recommended Movies:
                       title  genres
44                 Furious 7  Action
155   Fun with Dick and Jane  Comedy
162                  Stealth  Action
251       Gulliver's Travels  Comedy
321             The Campaign  Comedy
...                      ...     ...
4698       Hollywood Shuffle  Comedy
4716                  The FP  Comedy
4734            A True Story  Comedy
4746                  Clerks  Comedy
4752                 Slacker  Comedy

[303 rows x 2 columns]


In [1]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from flask import Flask, jsonify
import threading

# Muat model deteksi ekspresi wajah
model = load_model('emotion_model.h5')  # Pastikan Anda memiliki model deteksi ekspresi wajah

# Muat dataset movies.csv untuk rekomendasi
movies_df = pd.read_csv('emotion_service/dataset/processed_movies.csv')

# Daftar ekspresi yang dapat dikenali
emotion_labels = ['Angry', 'Disguist', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Muat model deteksi wajah
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Fungsi untuk merekomendasikan film berdasarkan ekspresi
def recommend_movies(emotion):
    recommendations = {
        'Happy': ['Comedy', 'Romantic', 'Action'],
        'Sad': ['Comedy', 'Drama', 'Romantic'],
        'Angry': ['Action', 'Thriller', 'Comedy'],
        'Fear': ['Horror', 'Mystery'],
        'Surprise': ['Fantasy', 'Action', 'Thriller'],
        'Neutral': ['Adventure', 'Animation'],
        'Disguist': ['Horror', 'Thriller']
    }

    genres = recommendations.get(emotion, ['General'])
    recommended_movies = movies_df[movies_df['genres'].isin(genres)]
    return recommended_movies[['title', 'genres']].to_dict(orient='records')

# Flask Setup
app = Flask(__name__)

@app.route('/detect_emotion', methods=['POST'])
def detect_emotion_from_face():
    # Capture image from webcam (or send image through API)
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        return jsonify({"error": "Failed to capture image"}), 400

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    if len(faces) == 0:
        return jsonify({"error": "No face detected"}), 400

    # Ambil wajah pertama yang terdeteksi
    (x, y, w, h) = faces[0]
    face = frame[y:y + h, x:x + w]
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face = cv2.resize(face, (48, 48))  # Mengubah ukuran sesuai model
    face = face / 255.0
    face = np.expand_dims(face, axis=0)

    # Prediksi ekspresi wajah
    emotion_prob = model.predict(face)
    max_index = np.argmax(emotion_prob[0])
    emotion = emotion_labels[max_index]

    # Mengambil rekomendasi film
    recommended_movies = recommend_movies(emotion)

    return jsonify({
        'emotion': emotion,
        'recommended_movies': recommended_movies
    })

# Fungsi untuk menjalankan Flask server di background
def run_flask():
    app.run(debug=True, use_reloader=False)

# Menjalankan Flask API di thread terpisah
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 17:24:43] "GET / HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step


INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 17:25:05] "POST /detect_emotion HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


...

INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 19:29:27] "POST /detect_emotion HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 19:29:31] "POST /detect_emotion HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 19:59:21] "POST /detect_emotion HTTP/1.1" 200 -
